In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date, timedelta
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

control_data = pd.read_csv('archive/control_group.csv', sep=';')
test_data = pd.read_csv('archive/test_group.csv', sep=';')

# Prepare the dataset (Garbage in, garbage out & quality data beats fancy algorithms)

## Have a look at both datasets

In [6]:
control_data.head()

,Campaign Name,Date,Spend (USD),Impressions,Reach,Website Clicks,Searches,Content Viewed,Added to cart,Purchase
0,Control Campaign,1.08.2019,2280,82702.000000,56930.000000,7016.000000,2290.000000,2159.000000,1819.0,618.000000
1,Control Campaign,2.08.2019,1757,121040.000000,102513.000000,8110.000000,2033.000000,1841.000000,1219.0,511.000000
2,Control Campaign,3.08.2019,2343,131711.000000,110862.000000,6508.000000,1737.000000,1549.000000,1134.0,372.000000
3,Control Campaign,4.08.2019,1940,72878.000000,61235.000000,3065.000000,1042.000000,982.000000,1183.0,340.000000
4,Control Campaign,5.08.2019,1835,109559.758621,88844.931034,5320.793103,2221.310345,1943.793103,1300.0,522.793103


In [3]:
test_data.head()

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Test Campaign,1.08.2019,3008,39550,35820,3038,1946,1069,894,255
1,Test Campaign,2.08.2019,2542,100719,91236,4657,2359,1548,879,677
2,Test Campaign,3.08.2019,2365,70263,45198,7885,2572,2367,1268,578
3,Test Campaign,4.08.2019,2710,78451,25937,4216,2216,1437,566,340
4,Test Campaign,5.08.2019,2297,114295,95138,5863,2106,858,956,768


## Changing the column names

In [3]:
control_data.columns = ['Campaign Name', 'Date', 'Spend (USD)', 'Impressions', 'Reach',
                         'Website Clicks', 'Searches', 'Content Viewed',
                         'Added to cart', 'Purchase']
test_data.columns = ['Campaign Name', 'Date', 'Spend (USD)', 'Impressions', 'Reach',
                         'Website Clicks', 'Searches', 'Content Viewed',
                         'Added to cart', 'Purchase']
test_data.columns

Index(['Campaign Name', 'Date', 'Spend (USD)', 'Impressions', 'Reach',
       'Website Clicks', 'Searches', 'Content Viewed', 'Added to cart',
       'Purchase'],
      dtype='object')

## Looking for null values

In [5]:
control_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Campaign Name   30 non-null     object 
 1   Date            30 non-null     object 
 2   Spend (USD)     30 non-null     int64  
 3   Impressions     30 non-null     float64
 4   Reach           30 non-null     float64
 5   Website Clicks  30 non-null     float64
 6   Searches        30 non-null     float64
 7   Content Viewed  30 non-null     float64
 8   Added to cart   30 non-null     float64
 9   Purchase        30 non-null     float64
dtypes: float64(7), int64(1), object(2)
memory usage: 2.5+ KB


## Another way

In [4]:
control_data.isnull().sum()

Campaign Name     0
Date              0
Spend (USD)       0
Impressions       0
Reach             0
Website Clicks    0
Searches          0
Content Viewed    0
Added to cart     0
Purchase          0
dtype: int64

In [4]:
test_data.isnull().sum()

Campaign Name          0
Date                   0
Spend [USD]            0
# of Impressions       0
Reach                  0
# of Website Clicks    0
# of Searches          0
# of View Content      0
# of Add to Cart       0
# of Purchase          0
dtype: int64

## There was null values in Impressions, Reach, Website Clicks, Searches, Content Viewed, Added to cart & Purchase columns and filled with the mean but the inplace paramater assigned to 'True' and this mean the changes implenemted in the original dataset.

In [2]:
control_data["# of Impressions"].fillna(control_data["# of Impressions"].mean(), inplace=True)
control_data["Reach"].fillna(control_data["Reach"].mean(), inplace=True)
control_data["# of Website Clicks"].fillna(control_data["# of Website Clicks"].mean(), inplace=True)
control_data["# of Searches"].fillna(control_data["# of Searches"].mean(), inplace=True)
control_data["# of View Content"].fillna(control_data["# of View Content"].mean(), inplace=True)
control_data["# of Add to Cart"].fillna(control_data["# of Add to Cart"].mean(), inplace=True)
control_data["# of Purchase"].fillna(control_data["# of Purchase"].mean(), inplace=True)

## Creating the ab_data(control_data + test_data)

In [7]:
ab_data = control_data.merge(test_data, how='outer').sort_values('Date')
ab_data.reset_index(drop=True)
ab_data

C:\Users\moham\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:1207: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  warnings.warn(


,Campaign Name,Date,Spend (USD),Impressions,Reach,Website Clicks,Searches,Content Viewed,Added to cart,Purchase
0,Control Campaign,1.08.2019,2280,82702.000000,56930.000000,7016.000000,2290.000000,2159.000000,1819.0,618.000000
30,Test Campaign,1.08.2019,3008,39550.000000,35820.000000,3038.000000,1946.000000,1069.000000,894.0,255.000000
39,Test Campaign,10.08.2019,2790,95054.000000,79632.000000,8125.000000,2312.000000,1804.000000,424.0,275.000000
9,Control Campaign,10.08.2019,2149,117624.000000,91257.000000,2277.000000,2475.000000,1984.000000,1629.0,734.000000
40,Test Campaign,11.08.2019,2420,83633.000000,71286.000000,3750.000000,2893.000000,2617.000000,1075.0,668.000000
10,Control Campaign,11.08.2019,2490,115247.000000,95843.000000,8137.000000,2941.000000,2486.000000,1887.0,475.000000
41,Test Campaign,12.08.2019,2831,124591.000000,10598.000000,8264.000000,2081.000000,1992.000000,1382.0,709.000000
11,Control Campaign,12.08.2019,2319,116639.000000,100189.000000,2993.000000,1397.000000,1147.000000,1439.0,794.000000
42,Test Campaign,13.08.2019,1972,65827.000000,49531.000000,7568.000000,2213.000000,2058.000000,1391.0,812.000000
12,Control Campaign,13.08.2019,2697,82847.000000,68214.000000,6554.000000,2390.000000,1975.000000,1794.0,766.000000


## Merging successfully completed?

In [8]:
ab_data['Campaign Name'].value_counts()

Control Campaign    30
Test Campaign       30
Name: Campaign Name, dtype: int64

## The analysis part to find the best campaign

## The relationship between the number of impressions and the amount spent on both campaigns

In [15]:
figure = px.scatter(data_frame = ab_data, 
                    x="Impressions",
                    y="Spend (USD)", 
                    size="Spend (USD)", 
                    color= "Campaign Name", 
                    trendline="ols")
figure.show()

## The control campaign resulted in more impressions

## The number of searches performed on the website from both campaigns

In [11]:
label = ["Total Searches from Control Campaign", 
         "Total Searches from Test Campaign"]
counts = [sum(control_data["Searches"]), 
          sum(test_data["Searches"])]
colors = ['gold','lightgreen']
fig = go.Figure(data=[go.Pie(labels=label, values=counts)])
fig.update_layout(title_text='Control Vs Test: Searches')
fig.update_traces(hoverinfo='label+percent', textinfo='value', 
                  textfont_size=30,
                  marker=dict(colors=colors, 
                              line=dict(color='black', width=3)))
# fig.show()

## The test campaign resulted in more searches on the website

##  The number of website clicks from both campaigns

In [12]:
label = ["Website Clicks from Control Campaign", 
         "Website Clicks from Test Campaign"]
counts = [sum(control_data["Website Clicks"]), 
          sum(test_data["Website Clicks"])]
colors = ['gold','lightgreen']
fig = go.Figure(data=[go.Pie(labels=label, values=counts)])
fig.update_layout(title_text='Control Vs Test: Website Clicks')
fig.update_traces(hoverinfo='label+percent', textinfo='value', 
                  textfont_size=30,
                  marker=dict(colors=colors, 
                              line=dict(color='black', width=3)))
# fig.show()

## The test campaign wins in the number of website clicks

## the amount of content viewed after reaching the website from both campaigns

In [13]:
label = ["Content Viewed from Control Campaign", 
         "Content Viewed from Test Campaign"]
counts = [sum(control_data["Content Viewed"]), 
          sum(test_data["Content Viewed"])]
colors = ['gold','lightgreen']
fig = go.Figure(data=[go.Pie(labels=label, values=counts)])
fig.update_layout(title_text='Control Vs Test: Content Viewed')
fig.update_traces(hoverinfo='label+percent', textinfo='value', 
                  textfont_size=30,
                  marker=dict(colors=colors, 
                              line=dict(color='black', width=3)))
# fig.show()

## The audience of the control campaign viewed more content than the test campaign. Although there is not much difference, as the website clicks of the control campaign were low, its engagement on the website is higher than the test campaign.

## The number of products added to the cart from both campaigns

In [17]:
label = ["Products Added to Cart from Control Campaign", 
         "Products Added to Cart from Test Campaign"]
counts = [sum(control_data["Added to cart"]), 
          sum(test_data["Added to cart"])]
colors = ['gold','lightgreen']
fig = go.Figure(data=[go.Pie(labels=label, values=counts)])
fig.update_layout(title_text='Control Vs Test: Added to Cart')
fig.update_traces(hoverinfo='label+percent', textinfo='value', 
                  textfont_size=30,
                  marker=dict(colors=colors, 
                              line=dict(color='black', width=3)))
# fig.show()

## Despite low website clicks more products were added to the cart from the control campaign.

## The amount spent on both campaigns

In [19]:
label = ["Amount Spent in Control Campaign", 
         "Amount Spent in Test Campaign"]
counts = [sum(control_data["Spend (USD)"]), 
          sum(test_data["Spend (USD)"])]
colors = ['gold','lightgreen']
fig = go.Figure(data=[go.Pie(labels=label, values=counts)])
fig.update_layout(title_text='Control Vs Test: Amount Spent')
fig.update_traces(hoverinfo='label+percent', textinfo='value', 
                  textfont_size=30,
                  marker=dict(colors=colors, 
                              line=dict(color='black', width=3)))
# fig.show()

## The amount spent on the test campaign is higher than the control campaign. But as we can see that the control campaign resulted in more content views and more products in the cart, the control campaign is more efficient than the test campaign.

## The purchases made by both campaigns

In [20]:
label = ["Purchases Made by Control Campaign", 
         "Purchases Made by Test Campaign"]
counts = [sum(control_data["Purchase"]), 
          sum(test_data["Purchase"])]
colors = ['gold','lightgreen']
fig = go.Figure(data=[go.Pie(labels=label, values=counts)])
fig.update_layout(title_text='Control Vs Test: Purchases')
fig.update_traces(hoverinfo='label+percent', textinfo='value', 
                  textfont_size=30,
                  marker=dict(colors=colors, 
                              line=dict(color='black', width=3)))
# fig.show()

## There’s only a difference of around 1% in the purchases made from both ad campaigns. As the Control campaign resulted in more sales in less amount spent on marketing, the control campaign wins here

# Which ad campaign converts more?

## The relationship between the number of website clicks and content viewed from both campaigns

In [22]:
figure = px.scatter(data_frame = ab_data, 
                    x="Content Viewed",
                    y="Website Clicks", 
                    size="Website Clicks", 
                    color= "Campaign Name", 
                    trendline="ols")
figure.show()

## The website clicks are higher in the test campaign, but the engagement from website clicks is higher in the control campaign. So the control campaign wins

## The relationship between the amount of content viewed and the number of products added to the cart from both campaigns

In [25]:
figure = px.scatter(data_frame = ab_data, 
                    x="Added to cart",
                    y="Content Viewed", 
                    size="Added to cart", 
                    color= "Campaign Name", 
                    trendline="ols")
figure.show()

## Again, the control campaign wins

## The number of products added to the cart and the number of sales from both campaigns

In [27]:
figure = px.scatter(data_frame = ab_data, 
                    x="Purchase",
                    y="Added to cart", 
                    size="Purchase", 
                    color= "Campaign Name", 
                    trendline="ols")
figure.show()

## Although the control campaign resulted in more sales and more products in the cart, the conversation rate of the test campaign is higher.

## From the above A/B tests, we found that the control campaign resulted in more sales and engagement from the visitors. More products were viewed from the control campaign, resulting in more products in the cart and more sales. But the conversation rate of products in the cart is higher in the test campaign. The test campaign resulted in more sales according to the products viewed and added to the cart. And the control campaign results in more sales overall. So, the Test campaign can be used to market a specific product to a specific audience, and the Control campaign can be used to market multiple products to a wider audience.